# Clustering Crypto

In [260]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [261]:
# Load the crypto_data.csv dataset.
# YOUR CODE HERE
df = pd.read_csv('Resources/files/crypto_data.csv')
df = df.rename(columns = {'Unnamed: 0':'Crypto'})
df.set_index(df.columns[0], inplace=True )

In [262]:
# Keep all the cryptocurrencies that are being traded.
# YOUR CODE HERE
df=df.loc[df.IsTrading == True]
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [263]:
# Keep all the cryptocurrencies that have a working algorithm.
# YOUR CODE HERE
df.dropna(subset=['Algorithm'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1144 entries, 42 to XBC
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 54.7+ KB


In [264]:
# Remove the "IsTrading" column. 
# YOUR CODE HERE
del df['IsTrading']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1144 entries, 42 to XBC
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   ProofType        1144 non-null   object 
 3   TotalCoinsMined  685 non-null    float64
 4   TotalCoinSupply  1144 non-null   object 
dtypes: float64(1), object(4)
memory usage: 53.6+ KB


In [265]:
# Remove rows that have at least 1 null value.
# YOUR CODE HERE
df.dropna(how="any", inplace=True)
df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [266]:
# Keep the rows where coins are mined.
# YOUR CODE HERE

#Assumption: Keep the rows where coins are mined means TotalCoinsMined > 0
df = df[df["TotalCoinsMined"] > 0]
df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [267]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE
coinNames_df = df[['CoinName']]

coinNames_df

,CoinName
Crypto,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex


In [268]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE
del df['CoinName']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 532 entries, 42 to XBC
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Algorithm        532 non-null    object 
 1   ProofType        532 non-null    object 
 2   TotalCoinsMined  532 non-null    float64
 3   TotalCoinSupply  532 non-null    object 
dtypes: float64(1), object(3)
memory usage: 20.8+ KB


In [269]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE
X = pd.get_dummies(df, columns=['Algorithm','ProofType'])
X.head(100)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Crypto,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XJO,3.919574e+07,45000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
XST,3.309373e+07,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ZCC,1.479902e+08,1000000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [270]:
# Standardize the data with StandardScaler().
X = StandardScaler().fit_transform(X)
X


array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [271]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X)
crypto_pca


array([[-0.32941155,  1.04766943, -0.57563565],
       [-0.31274903,  1.04781695, -0.57594579],
       [ 2.30602597,  1.6478538 , -0.65982577],
       ...,
       [ 0.32471357, -2.31613749,  0.42185154],
       [-0.14878806, -2.11029773,  0.42661078],
       [-0.2838562 ,  0.81259216, -0.25802107]])

In [272]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE
colNames = ['PC 1','PC 2','PC 3']
pcs_df = pd.DataFrame(crypto_pca, columns=colNames, index=df.index)
print(pcs_df)

            PC 1      PC 2      PC 3
Crypto                              
42     -0.329412  1.047669 -0.575636
404    -0.312749  1.047817 -0.575946
1337    2.306026  1.647854 -0.659826
BTC    -0.150614 -1.286942  0.190932
ETH    -0.153596 -2.004598  0.361390
...          ...       ...       ...
ZEPH    2.469658  0.877977  0.043744
GAP    -0.327457  1.047550 -0.575655
BDX     0.324714 -2.316137  0.421852
ZEN    -0.148788 -2.110298  0.426611
XBC    -0.283856  0.812592 -0.258021

[532 rows x 3 columns]


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [273]:
# Create an elbow curve to find the best value for K.
kList = list(range(1, 11))
inertia = []
for i in kList:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

elbow_data = {"k":kList, "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x = "k", y="inertia", xticks=kList, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`


In [274]:
# Initialize the K-Means model.
k = 4
km = KMeans(n_clusters=k, random_state=0)

# Fit the model
km.fit(pcs_df)


# Predict clusters
predictions = km.predict(pcs_df)
predictions

array([0, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 0, 3, 0, 3, 3, 0, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 0, 3, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0,
       3, 3, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 0, 3, 0, 0, 3, 3, 3, 3, 0,
       0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 0, 3, 0, 0, 3, 0, 3, 0, 3,
       0, 3, 0, 0, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 0,
       3, 0, 3, 3, 0, 3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 3, 0, 0,
       0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 0, 3, 0, 3, 0, 3, 0, 3, 0, 0, 0, 0,
       3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 0,
       0, 3, 0, 3, 3, 0, 3, 3, 0, 3, 0, 0, 0, 3, 0, 0, 3, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 0, 3,
       0, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 3, 0, 3, 0,

In [275]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = coinNames_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
Crypto,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.329412,1.047669,-0.575636,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.312749,1.047817,-0.575946,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.306026,1.647854,-0.659826,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.150614,-1.286942,0.190932,Bitcoin,3
ETH,Ethash,PoW,1.076842e+08,0,-0.153596,-2.004598,0.361390,Ethereum,3
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.159931,-1.119762,-0.018317,Litecoin,3
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.395987,1.241614,-0.489473,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.154275,-2.237726,0.416666,Monero,3
ETC,Ethash,PoW,1.133597e+08,210000000,-0.152038,-2.004701,0.361375,Ethereum Classic,3


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [278]:
# Creating a 3D-Scatter with the PCA data and the clusters
# Plotting the clusters with three features

CoinNames = list(clustered_df["CoinName"])
Algorithms = list(clustered_df['Algorithm'])

fig = px.scatter_3d(clustered_df, x="PC 1", y="PC 2", z="PC 3", color="Class", symbol="Class",width=800, hover_data = ['Algorithm'], hover_name = "CoinName" )
fig.update_layout(legend=dict(x=0,y=1))
fig.show()


In [286]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName","Algorithm","ProofType","TotalCoinSupply","TotalCoinsMined","Class"], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [287]:
# Print the total number of tradable cryptocurrencies.
print(f" Total Number of Tradable Cryptocurriences = {len(clustered_df)} ")

 Total Number of Tradable Cryptocurriences = 532 


In [292]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
mScale=MinMaxScaler()
graph_arr=mScale.fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
graph_df = pd.DataFrame(graph_arr)
graph_df
##########################################################################################
#here I am 
##############################

,0,1
0,4.200000e-11,0.000000e+00
1,5.320000e-04,1.065855e-03
2,3.141593e-01,2.957551e-02
3,2.100000e-05,1.810842e-05
4,0.000000e+00,1.087731e-04
...,...,...
527,2.000000e-03,2.020225e-03
528,2.500000e-04,1.508199e-05
529,1.400223e-03,9.901351e-04
530,2.100000e-05,7.370282e-06


In [235]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.




# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

NameError: name 'plot_df' is not defined

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
